# Анализ-исследование надёжности клиентов кредитного отдела банка

Заказчик — крупный банк, кредитный отдел. Наша задача - выяснить, влияют ли такие факторы, как семейное положение, количество детей клиента на способность погашения кредита в срок. Входные данные от банка — основные данные о клиентах банка, такие включая данные о наличии задолженности, трудоустройстве, образовании и прочем.

Результаты данного исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. Также необходимо ответить на вопросы: Есть ли зависимость между наличием детей, семейным положением, уровнем дохода и возвратом кредита в срок? Как разные цели кредита влияют на его возврат?

## Загрузка данных и изучение общей информации

In [26]:
import pandas as pd
from pymystem3 import Mystem # импорт библиотеки для лемматизации

data = pd.read_csv('/datasets/data.csv') # выгружаем базу данных в переменную
data.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Вывел на экран первые десять строк таблицы. Теперь нужно получить общую информацию о таблице:

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице двенадцать столбцов.

Согласно документации к данным:
* children — количество детей в семье
* days_employed — трудовой стаж в днях  
* dob_days — возраст клиента в годах
* education — образование клиента
* education_id — идентификатор образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — доход в месяц
* purpose — цель получения кредита

Количество значений в столбцах days_employed и total_income отличается от других столбцов. 


**Вывод**

В каждой строке таблицы — данные о каждом заёмщике. 

Предварительно можно утверждать, что, данных достаточно для проверки, но они требуют обработки. Встречаются пропуски, а также различия в регистре. Необходимо разобраться, что означают отрицательные значения в столбце days_employed, ошибка либо вид предоставления.

## Предобработка данных

### Обработка пропусков

Имеются пропуски в столбцах days_employed и total_income. Происхождение пропусков неясно, возможно произошла утеря данных.

In [28]:
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


Общий доход и стаж это количественные переменные, их лучше заменить медианными значениями.
Заменим пропущенные значения общего дохода на медианные значения в каждом крупном виде занятости (пропустим три незначительно представленных вида).

In [29]:
for typ in ['сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'предприниматель']:#цикл для прохождения по видам занятости
    med = data[data['income_type']==typ]['total_income'].median()# нахождение медианы дохода в данном виде занятости
    for i in range(len(data)):
        if (data.loc[i, 'income_type']==typ) & (pd.isna(data.loc[i, 'total_income'])):# при наличии пропуска в этом виде занятости 
            data.loc[i, 'total_income'] = med #  - заполнение соответствующей медианой
data['total_income'].isna().sum() # проверка на пропуски


0

Далее для них же заменим пропущенные значения в столбце days_employed на медианы. Но для начала надо разобраться с категорией "пенсионеры": введены некорректные данные, у каждого больше 300000 дней - явно нереальное число. Похоже, что данные введены в часах, если разделить значения у пенсионеров на 24, получатся приемлемые данные. Отрицательные значения тоже встречаются, скорее всего изза неверного ввода и просчёта их системой - если дата приёма и увольнения с работы были перепутаны местами.
Также надо исправить отрицательные значения методом abs().

In [30]:
for i in range(len(data)):
    if data.loc[i, 'income_type'] == 'пенсионер':
        data.loc[i, 'days_employed'] = (data.loc[i, 'days_employed'])/24 #поделил стаж у пенсионеров на 24, если предположить, что он был в часах

data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median()).abs()# заполнение пропусков и приведение к положительным значениям.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Теперь все значительные пропуски обработаны. Значения стажа в категории "пенсионеры" поправлены, а отрицательные значения иправлены на положительные.

### Замена типа данных

In [31]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

1) Для лучшего восприятия в стобце days_employed надо заменить тип данных на целочисленный.
2) В столбце total_income доход округлить до 2 знаков после запятой
3) В столбце children избавимся от отрицательных значений.

In [32]:
data['days_employed'] = data['days_employed'].astype('int') # надо заменить тип данных на целочисленный
data['total_income'] = data['total_income'].round(decimals=2) # доход округлить до 2 знаков после запятой
data['children'] = data['children'].abs()

In [33]:
data['dob_years'].unique() #проверим аномальные значения в столбце 'dob_years'

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Имеется аномальное значение возраста 0 лет, необходимо посмотреть количество значений в датасете, чтобы решить, что с ними делать.

In [34]:
data[data['dob_years']==0].count()

children            101
days_employed       101
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income        101
purpose             101
dtype: int64

101 клиентов со значением возраста 0 довольно много, можно заменить значение возраста на медиану.

In [35]:
for i in range(len(data)):
    if data.loc[i, 'dob_years']==0: # при возрасте 0 лет 
        data.loc[i, 'dob_years'] = data['dob_years'].median()
data[data['dob_years']==0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Изучим столбец gender:

In [36]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Здесь всего аномальное значение в графе пол - XNA, так как это несущественная характеристика для нашего исследования и значение одно, делать с ним ничего не будем.

**Вывод**

Данные обработаны, теперь они лучше воcпринимаются.

### Обработка дубликатов

Сначала нужно найти полные дубликаты строк и удалить их, так как вряд ли найдётся два разных человека с полностью идентичными данными во всех стобцах.
Затем необходимо изучить категориальные столбцы education, family_status (столбец income_type был изучен на этапе удаления пропусков). Данные могут содержать неявные дубликаты, введены с разным регистром и прочее.

In [37]:
print('Всего явных дубликатов: ', data.duplicated().sum())# находим количество явных дубликатов строк. 
print(data['education'].unique()) # проверяем значения на неявные дубликаты
print(data['family_status'].unique()) # проверяем значения на неявные дубликаты


Всего явных дубликатов:  55
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


In [38]:
data = data.drop_duplicates().reset_index(drop=True) # Удаляем явные дубликаты из таблицы.
print('Всего явных дубликатов: ', data.duplicated().sum())

Всего явных дубликатов:  0


В стобце family_status дубликатов нет, а стоблец education надо привести к одному регистру.

In [39]:
data['education'] = data['education'].str.lower()
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу


**Вывод**

Удалены явные и неявные дубликаты в данных. Данные в стобцах приведены к одному регистру, в категориальных переменных не встречаются дубликаты, кроме последнего столбца.

### Лемматизация

Изучим значения в колонке purpose прежде чем проводить лемматизацию.

In [40]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Здесь имеем четыре основных целей кредита - жильё, образование, свадьба, недвижимость (жилая и нежилая). Проведём лемматизацию для значений в столбце purpose, и выведем из них главные слова, обозначающие цель кредита, выведем их в отдельный столбец purpose_group

In [41]:
m = Mystem()
def lemming(string):
    lemmas = ' '.join(m.lemmatize(string)) # для соединения выделенных лемм в строки, в которых мы будем искать слова
    if 'образование' in lemmas: # выделяем 5 категорий - целей кредита, по ключевым словам
        return 'образование' 
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'автомобил' in lemmas:
        return 'автомобиль'
    if 'жил' in lemmas:
        return 'жилая недвижимость'
    return 'недвижимость'

data['purpose_group'] = data['purpose'].apply(lemming)

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,жилая недвижимость
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,жилая недвижимость
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,образование
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,свадьба
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,покупка жилья,жилая недвижимость
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,операции с жильем,жилая недвижимость
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823.93,образование,образование
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856.83,на проведение свадьбы,свадьба
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,покупка жилья для семьи,жилая недвижимость


**Вывод**

Лемматизация прошла успешно, но с недвижимостью ясно всё не на сто процентов. Иногда неясно, какая именно недвижимость, жилая или нежилая указывана графе цель кредита, вся без указания "жилая" отнесена просто в недвижимость.

### Категоризация данных

У нас имеется два столбца - education_id и family_status_id, содержащие краткие идентификаторы для других столбцов - education и family_status соответственно. Можно выделить словари в данных.
Фактически провели категоризацию для данных целей кредита, осталось провести категоризацию по уровню дохода, возрасту, количеству детей.

In [42]:
education_dict=data[['education','education_id']] #создание словаря по уровню образования
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict.sort_values(by='education_id'))

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


In [43]:
family_status_dict=data[['family_status','family_status_id']] #создание словаря по семейному положению
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict.sort_values(by='family_status_id'))

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


In [44]:
def income_group(row): # создаём функцию для категоризации по уровню дохода
    if row > 200000:
        return 'A'
    if 143000 <= row <= 200000:
        return 'B'
    if 108000 <= row < 143000:
        return 'C'
    return 'D'

data['income_group'] = data['total_income'].apply(income_group)
print(data.groupby('income_group')['debt'].count())

income_group
A    5067
B    5501
C    5491
D    5411
Name: debt, dtype: int64


In [45]:
def children_group(row): # создаём функцию для категоризации по количеству детей
    if row >= 3:
        return 'многодетные'
    if 1 < row < 3:
        return '2-3 ребёнка'
    if 0 < row <= 1:
        return '1 ребёнок'
    return 'без детей'

data['children_group'] = data['children'].apply(children_group)

Далее вынесем отдельную таблицу для анализа, только с необходимыми для анализа столбцами:

In [46]:
data_analysis = data[['education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'purpose_group', 'income_group', 'children_group']]
data_analysis.head(10)

,education_id,family_status_id,gender,income_type,debt,purpose_group,income_group,children_group
0,0,0,F,сотрудник,0,жилая недвижимость,A,1 ребёнок
1,1,0,F,сотрудник,0,автомобиль,C,1 ребёнок
2,1,0,M,сотрудник,0,жилая недвижимость,B,без детей
3,1,0,M,сотрудник,0,образование,A,многодетные
4,1,1,F,пенсионер,0,свадьба,B,без детей
5,0,1,M,компаньон,0,жилая недвижимость,A,без детей
6,0,0,F,компаньон,0,жилая недвижимость,A,без детей
7,1,0,M,сотрудник,0,образование,C,без детей
8,0,1,F,сотрудник,0,свадьба,D,2-3 ребёнка
9,1,0,M,сотрудник,0,жилая недвижимость,B,без детей


**Вывод**

Выделены словари для категорий по семейному статусу и уровню образования. Созданы отдельные категории по уровню дохода, количеству и наличию детей.

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [47]:
data_analysis.groupby('children_group')['debt'].value_counts() # смотрим зависимость задолженности от наличия детей.

children_group  debt
1 ребёнок       0        4411
                1         445
2-3 ребёнка     0        1858
                1         194
без детей       0       13043
                1        1063
многодетные     0         417
                1          39
Name: debt, dtype: int64

**Вывод**

Процент должников в категории без детей практически такой же, как и у категорий с детьми - ниже на один-два процентных пункта. В целом отличия несущественны.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [48]:
data.groupby('family_status')['debt'].value_counts() # смотрим зависимость задолженности от семейного положения

family_status          debt
Не женат / не замужем  0        2536
                       1         274
в разводе              0        1110
                       1          85
вдовец / вдова         0         896
                       1          63
гражданский брак       0        3774
                       1         388
женат / замужем        0       11413
                       1         931
Name: debt, dtype: int64

**Вывод**

Процент должников в категориях "Не женат / не замужем" и "гражданский брак" немногим выше, чем у остальных категорий, но незначительно.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
data_analysis.groupby('income_group')['debt'].value_counts()  # смотрим зависимость задолженности от уровня дохода

income_group  debt
A             0       4709
              1        358
B             0       5034
              1        467
C             0       5006
              1        485
D             0       4980
              1        431
Name: debt, dtype: int64

**Вывод**

Если смотреть на процент задолженностей в категориях по доходам, видно, что самый низкий процент задолженности у категории высоких доходов. У категорий средних доходов самый большой процент должников, но лишь немногим выше.

- Как разные цели кредита влияют на его возврат в срок?

In [50]:
data_analysis.groupby('purpose_group')['debt'].value_counts() # смотрим зависимость задолженности от целей кредита

purpose_group       debt
автомобиль          0       3905
                    1        403
жилая недвижимость  0       5295
                    1        397
недвижимость        0       4737
                    1        385
образование         0       3644
                    1        370
свадьба             0       2148
                    1        186
Name: debt, dtype: int64

**Вывод**

Самый большой процент по задолженностям тут - это кредиты на автомобиль и образование. Самый низкий - на жилую недвижимость.

## Общий вывод

   Был проведён анализ базы данных статистики о платёжеспособности клиентов. Необходимо было разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
База данных сначала была изучена на предмет структуры, типов значений, ошибок, дубликатов и аномальных значений.
Явные дубликаты значений были удалены, неявные дубликаты обработаны, отрицательные значения исправлены, пропущенные значения приведены к средним и медианным.
Была проведена лемматизация для стоблца графы "цель кредита" с целью последующего выделения ключевых целей, для которых берётся заём.
Проведена категоризация данных по ключевым параметрам дохода, образования, семейного положения и количества детей клиентов для последующего анализа.
Проведён анализ, даны ответы на ключевые вопросы, выяснено, что общий уровень задолженностей в разных категориях по уровню дохода, семейному положению, количеству детей, целям кредита примерно находится на одном уровне, отличаясь незначительно.
Для последующего анализа данных клиентов рекомендация кредитному отделу банка поработать над улучшением заполняемости базы данных, а точнее:
добавлять в базу данных данные о текущих имеющихся заёмах в других банках, так как общая долговая нагрузка влияет на вероятность погашения кредита в срок; поработать корректной заполняемостью уже имеющихся полей.
